In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == 'notebook':
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from preprocessing import (
    load_raw_data,
    clean_data,
    add_features,
    encode_categoricals,
    split_features_target,
)


In [ ]:
# Import opzionali (XGBoost/SHAP)
try:
    from xgboost import XGBRegressor
    XGBOOST_AVAILABLE = True
except Exception as e:
    XGBOOST_AVAILABLE = False
    XGBRegressor = None
    print(f'XGBoost non disponibile: {e}')

try:
    import shap
    SHAP_AVAILABLE = True
except Exception as e:
    SHAP_AVAILABLE = False
    shap = None
    print(f'SHAP non disponibile: {e}')


In [ ]:
#Caricamento del dataset
df = load_raw_data()

#Anteprima delle prime righe del dataset
df.head()


In [ ]:
#Informazioni generali: num righe e colonne, tipi di variabili, valori null
df.info()

In [ ]:
#Statistiche relative alle variabili numeriche
df.describe()

In [ ]:
#Analisi dei valori mancanti con ordinamento colonne per numero di NaN
df.isnull().sum().sort_values(ascending=False).head(20)

In [ ]:
#Matrice di correlazione
corr = df.corr(numeric_only=True)
#Visualizzazione delle variabili correlate tra loro
plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap="coolwarm")
plt.show()

In [ ]:
#PULIZIA DATI E IMPUTAZIONE VALORI MANCANTI
df = clean_data(df)

#FEATURE ENGINEERING
df = add_features(df)


In [ ]:
#Controllo valori mancanti residui
df.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:
#Num totale di valori mancanti nel dataset
df.isnull().sum().sum()

In [ ]:
#Statistiche post-pulizia
df.describe()

In [ ]:
#Istogramma per la distribuzione della variabile target SalePrice
sns.histplot(df["SalePrice"], kde=True)
plt.title("Distribuzione di SalePrice")
plt.show()

In [ ]:
#Analisi correlazione tra variabili numeriche e SalePrice
corr = df.corr(numeric_only=True)["SalePrice"].sort_values(ascending=False)
corr.head(15)

In [ ]:
#Codifica variabili categoriche in numeriche
df_encoded = encode_categoricals(df, drop_first=True)

#Separazione tra feature (X) e target (y)
X, y = split_features_target(df_encoded, target='SalePrice', log_target=True)

df_encoded.head()


In [ ]:
# TRAINING PIPELINE
from models.train_pipeline import train_and_evaluate

results_df, trained_models, (X_train, X_test, y_train, y_test) = train_and_evaluate(
    use_xgboost=XGBOOST_AVAILABLE,
    save_best=True,
    return_data=True
)

results_df

# Modello XGBoost (se disponibile)
xgb = trained_models.get('XGBoost')
if xgb is None:
    print('XGBoost non disponibile o non addestrato.')


In [ ]:
# PREDIZIONE SU ESEMPIO Casa 1
if not XGBOOST_AVAILABLE or 'xgb' not in globals() or xgb is None:
    print('XGBoost non disponibile, salto questa cella.')
else:


    # Selezioniamo la prima casa del test set
    sample = X_test.iloc[0:1]
    pred_log = xgb.predict(sample)
    pred_price = np.expm1(pred_log)
    pred_price

In [ ]:
# Visualizziamo le feature della casa per interpretare la predizione
X_test.iloc[0:1].T

In [ ]:
# PREZZO REALE DELLA PRIMA CASA DEL TEST SET
true_log_price = y_test.iloc[0]
true_price = np.expm1(true_log_price)
true_price

In [ ]:
# IMPORTANZA DELLE FEATURE SECONDO XGBOOST
if not XGBOOST_AVAILABLE or 'xgb' not in globals() or xgb is None:
    print('XGBoost non disponibile, salto questa cella.')
else:

    import pandas as pd

    # Calcolo dell'importanza delle feature dal modello
    importance = xgb.get_booster().get_score(importance_type='gain')

    importance_df = pd.DataFrame({
        'Feature': list(importance.keys()),
        'Importance': list(importance.values())
    })

    # Se XGBoost usa feature index (f0, f1, ...), mappa agli effettivi nomi colonna
    if importance_df['Feature'].str.match(r'^f\d+$').all():
        importance_df['Index'] = importance_df['Feature'].str.extract(r'^f(\d+)$').astype(int)
        importance_df['FeatureName'] = importance_df['Index'].apply(lambda i: X.columns[i])
    else:
        # Altrimenti le feature sono già nominate
        importance_df['FeatureName'] = importance_df['Feature']

    # Ordina per importanza
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    importance_df


In [ ]:
# GRAFICO DELLE TOP 20 FEATURE
plt.figure(figsize=(10, 12))
plt.barh(importance_df['FeatureName'].head(20), importance_df['Importance'].head(20))
plt.gca().invert_yaxis()
plt.title("Top 20 Feature Importance (Gain)")
plt.show()

In [ ]:
# Installazione/aggiornamento di ipywidgets (opzionale)
# !pip install ipywidgets --upgrade


In [ ]:
# SHAP (TreeExplainer)
if not SHAP_AVAILABLE or 'xgb' not in globals() or xgb is None:
    print('SHAP o XGBoost non disponibile, salto questa cella.')
else:
    explainer = shap.TreeExplainer(xgb)

    # Calcoliamo i valori SHAP per il test set
    shap_values = explainer.shap_values(X_test)


In [ ]:
# FORCE PLOT per la prima casa del test set
if not SHAP_AVAILABLE or 'shap_values' not in globals() or 'explainer' not in globals():
    print('SHAP non disponibile o valori non calcolati, salto questa cella.')
else:
    shap.force_plot(
        explainer.expected_value,
        shap_values[0,:],
        X_test.iloc[0,:],
        matplotlib=True
    )


In [ ]:
# PREDIZIONE SU ESEMPIO Casa 2
if not XGBOOST_AVAILABLE or 'xgb' not in globals() or xgb is None:
    print('XGBoost non disponibile, salto questa cella.')
else:

    sample = X_test.iloc[1:2]
    pred_log = xgb.predict(sample)
    pred_price = np.expm1(pred_log)
    pred_price

In [ ]:
# Visualizziamo le feature della casa per interpretare la predizione
X_test.iloc[1:2].T

In [ ]:
# PREZZO REALE DELLA SECONDA CASA DEL TEST SET
true_log_price = y_test.iloc[1]
true_price = np.expm1(true_log_price)
true_price

In [ ]:
# FORCE PLOT SHAP PER LA SECONDA CASA
if not SHAP_AVAILABLE or 'shap_values' not in globals() or 'explainer' not in globals():
    print('SHAP non disponibile o valori non calcolati, salto questa cella.')
else:
    shap.force_plot(
        explainer.expected_value,
        shap_values[1,:],
        X_test.iloc[1,:],
        matplotlib=True
    )


In [ ]:
# FORCE PLOT PER CASA 1 E CASA 2
if not SHAP_AVAILABLE or 'shap_values' not in globals() or 'explainer' not in globals():
    print('SHAP non disponibile o valori non calcolati, salto questa cella.')
else:
    # Casa 1
    print('Casa 1')
    shap.force_plot(
        explainer.expected_value,
        shap_values[0,:],
        X_test.iloc[0,:],
        matplotlib=True
    )

    # Casa 2
    print('Casa 2')
    shap.force_plot(
        explainer.expected_value,
        shap_values[1,:],
        X_test.iloc[1,:],
        matplotlib=True
    )


In [ ]:
# CONFRONTO SHAP TRA LE DUE CASE
if not SHAP_AVAILABLE or 'shap_values' not in globals() or 'explainer' not in globals():
    print('SHAP non disponibile o valori non calcolati, salto questa cella.')
else:
    import pandas as pd
    import matplotlib.pyplot as plt

    # Indici delle due case
    i1 = 0
    i2 = 1

    # Conversione dei valori SHAP in serie Pandas
    shap1 = pd.Series(shap_values[i1], index=X_test.columns)
    shap2 = pd.Series(shap_values[i2], index=X_test.columns)

    # Selezione delle 10 feature più influenti per ciascuna casa
    top1 = shap1.abs().sort_values(ascending=False).head(10)
    top2 = shap2.abs().sort_values(ascending=False).head(10)

    # Unione delle feature più importanti
    features = list(set(top1.index) | set(top2.index))

    # Dataframe confronto
    df_compare = pd.DataFrame({
        'Casa 1': shap1[features],
        'Casa 2': shap2[features]
    })

    # Grafico comparativo
    plt.figure(figsize=(10, 8))
    df_compare.plot(kind='barh', figsize=(12, 10))
    plt.title('Confronto SHAP tra Casa 1 e Casa 2')
    plt.xlabel('SHAP value (impatto sul prezzo)')
    plt.show()


In [ ]:
import os

os.makedirs("../data/processed", exist_ok=True)

In [ ]:
df.to_csv("../data/processed/cleaned_data.csv", index=False)

In [ ]:
df_encoded.to_csv("../data/processed/df_encoded.csv", index=False)
print("df_encoded.csv salvato in data/processed/")

## Conclusioni Finali

Il progetto aveva l’obiettivo di costruire un modello in grado di prevedere il prezzo delle case nel dataset Ames Housing.
Attraverso un processo completo di analisi, pulizia, feature engineering e modellazione, siamo arrivati a un risultato solido e interpretabile.

### Prestazioni dei modelli
Sono stati confrontati diversi algoritmi di regressione:

- **Linear Regression** (baseline)
- **Random Forest**
- **Gradient Boosting**
- **XGBoost**

Il modello con le migliori prestazioni è risultato essere **XGBoost**, che ha ottenuto l’RMSE più basso sul test set, dimostrando una capacità superiore nel catturare relazioni non lineari e interazioni tra le variabili.

### Feature più importanti
Dall’analisi dell’importanza delle feature e dai valori SHAP, emergono come più influenti:

- **OverallQual** (qualità generale della casa)
- **GrLivArea** (superficie abitabile sopra il livello del suolo)
- **TotalSF** (superficie totale, creata tramite feature engineering)
- **GarageCars** (capacità del garage)
- **YearBuilt / HouseAge** (età della casa)
- **Neighborhood** (zona di appartenenza)

Queste variabili risultano determinanti nel definire il valore di mercato di un’abitazione.

### Interpretazione con SHAP
L’utilizzo di SHAP ha permesso di:

- comprendere come ogni feature contribuisce alla predizione del modello
- analizzare singole case tramite force plot
- confrontare due abitazioni e osservare differenze nei fattori che aumentano o diminuiscono il prezzo

Questo rende il modello non solo accurato, ma anche **interpretabile**, un aspetto fondamentale in ambito immobiliare.

### Considerazioni finali
Il progetto dimostra che:

- un’adeguata fase di preprocessing e feature engineering migliora significativamente le prestazioni
- modelli avanzati come XGBoost offrono ottimi risultati su dati strutturati
- strumenti di interpretabilità come SHAP sono essenziali per comprendere e comunicare le decisioni del modello

### Possibili sviluppi futuri
- Ottimizzazione degli iperparametri tramite Grid Search o Bayesian Optimization
- Aggiunta di nuove feature derivate (es. interazioni tra variabili)
- Validazione incrociata per una stima più robusta delle prestazioni
- Sviluppo di un’interfaccia (web o CLI) per effettuare predizioni su nuove case

Il modello finale rappresenta una base solida per un sistema di valutazione immobiliare accurato e interpretabile.